In [1]:
import pandas as pd
import numpy as np

In [2]:
pcd_df = pd.read_excel('PCD_ImageDepthFusion.xlsx')
annotations_df = pd.read_excel('Annotations.xlsx')

In [3]:
print(pcd_df.shape)
print(annotations_df.shape)

(24318, 30)
(18538, 20)


In [4]:
pcd_df.head(2)

,sample_token,x_global,y_global,z_global,radar_token,radar_x,radar_y,radar_z,radar_cam_x,radar_cam_y,...,x_rms,y_rms,invalid_state,pdh0,vx_rms,vy_rms,image_coord_x,image_coord_y,depth,depth_patch
0,ca9a282c9e77460f8360f564131a8af5,412.256438,1168.024119,0.256821,37091c75b9704e0daa829ba56dfa0906,8.2,5.3,0,-5.254447,0.959718,...,19,19,0,1,17,3,176.668179,608.328984,19.449539,[[18.6341 18.6538 18.6734 18.6925 18.7115 18.7...
1,ca9a282c9e77460f8360f564131a8af5,410.837981,1167.050491,0.262282,37091c75b9704e0daa829ba56dfa0906,9.6,4.3,0,-4.251772,0.954851,...,19,19,0,1,16,3,360.028420,593.967793,19.042000,[[19.0424 19.0348 19.0262 19.0175 19.0075 18.9...


In [5]:
annotations_df.head(2)

,scene_name,scene_token,sample_token,timestamp,annotation_token,category,instance_token,center_x,center_y,center_z,length,width,height,rotation_w,rotation_x,rotation_y,rotation_z,yaw,num_lidar_pts,num_radar_pts
0,scene-0061,cc8c0bf57f984915a77078b10eb33198,ca9a282c9e77460f8360f564131a8af5,1532402927647951,ef63a697930c4b20a6b9791f423351da,human.pedestrian.adult,6dd2cbf4c24b4caeb625035869bca7b5,373.256,1130.419,0.800,0.621,0.669,1.642,0.983110,0,0,-0.183016,-0.368107,1,0
1,scene-0061,cc8c0bf57f984915a77078b10eb33198,ca9a282c9e77460f8360f564131a8af5,1532402927647951,6b89da9bf1f84fd6a5fbe1c3b236f809,human.pedestrian.adult,48d58b69b40149aeb2e64aa4b1a9192f,378.888,1153.348,0.865,0.775,0.769,1.711,-0.552759,0,0,0.833341,-1.970250,2,0


In [6]:
merged_df = pd.merge(pcd_df, annotations_df, on='sample_token', suffixes=('_pcd', '_ann'))
merged_df.shape

(1126924, 49)

In [7]:
merged_df.columns.to_list()

['sample_token',
 'x_global',
 'y_global',
 'z_global',
 'radar_token',
 'radar_x',
 'radar_y',
 'radar_z',
 'radar_cam_x',
 'radar_cam_y',
 'radar_cam_z',
 'dyn_prop',
 'cluster_id',
 'rcs',
 'vx',
 'vy',
 'vx_comp',
 'vy_comp',
 'is_quality_valid',
 'ambig_state',
 'x_rms',
 'y_rms',
 'invalid_state',
 'pdh0',
 'vx_rms',
 'vy_rms',
 'image_coord_x',
 'image_coord_y',
 'depth',
 'depth_patch',
 'scene_name',
 'scene_token',
 'timestamp',
 'annotation_token',
 'category',
 'instance_token',
 'center_x',
 'center_y',
 'center_z',
 'length',
 'width',
 'height',
 'rotation_w',
 'rotation_x',
 'rotation_y',
 'rotation_z',
 'yaw',
 'num_lidar_pts',
 'num_radar_pts']

In [8]:
merged_df['distance'] = np.sqrt(
    (merged_df['x_global'] - merged_df['center_x'])**2 +
    (merged_df['y_global'] - merged_df['center_y'])**2 +
    (merged_df['z_global'] - merged_df['center_z'])**2
)

In [9]:
def filter_by_distance(cut_off):
    return merged_df[merged_df['distance'] < cut_off].copy()

final_df = filter_by_distance(2.0)
print(final_df.shape)

(3860, 50)


In [10]:
caninfo_df = pd.read_excel('caninfo.xlsx')
caninfo_df.head(2)

,scene_name,scene_token,sample_token,timestamp,can_timestamp,brake,brake_switch,rear_left_rpm,rear_right_rpm,can_utime,vehicle_speed,yaw_rate
0,scene-0061,cc8c0bf57f984915a77078b10eb33198,ca9a282c9e77460f8360f564131a8af5,1532402927647951,1532402928127800,0,1,275.2617,276.1791,1532402928127800,31.44,0.6
1,scene-0061,cc8c0bf57f984915a77078b10eb33198,39586f9d59004284a7114a68825e8eec,1532402928147847,1532402928127800,0,1,275.2617,276.1791,1532402928127800,31.44,0.6


In [11]:
final_df = final_df.merge(
    caninfo_df,
    on=['scene_name', 'scene_token', 'sample_token', 'timestamp'],
    how='left',
    suffixes=('', '_can')
)
final_df.head(2)

,sample_token,x_global,y_global,z_global,radar_token,radar_x,radar_y,radar_z,radar_cam_x,radar_cam_y,...,num_radar_pts,distance,can_timestamp,brake,brake_switch,rear_left_rpm,rear_right_rpm,can_utime,vehicle_speed,yaw_rate
0,ca9a282c9e77460f8360f564131a8af5,400.178422,1170.502939,0.495531,37091c75b9704e0daa829ba56dfa0906,10.0,-6.9,0,6.948967,0.96407,...,0,1.239830,1532402928127800,0,1,275.2617,276.1791,1532402928127800,31.44,0.6
1,ca9a282c9e77460f8360f564131a8af5,400.178422,1170.502939,0.495531,37091c75b9704e0daa829ba56dfa0906,10.0,-6.9,0,6.948967,0.96407,...,0,1.689902,1532402928127800,0,1,275.2617,276.1791,1532402928127800,31.44,0.6


In [12]:
sample_pcd_image_ref_df = pd.read_excel('Sample-PCD-ImageFileReference.xlsx')
sample_pcd_image_ref_df.head(2)

,scene_token,scene_name,sample_token,next_token,radar_pcd_file,image_file,camera_pose_x,camera_pose_y,camera_pose_z
0,cc8c0bf57f984915a77078b10eb33198,scene-0061,ca9a282c9e77460f8360f564131a8af5,39586f9d59004284a7114a68825e8eec,../data/sets/nuscenes\samples/RADAR_FRONT/n015...,../data/sets/nuscenes\samples/CAM_FRONT/n015-2...,411.419986,1181.197176,0
1,cc8c0bf57f984915a77078b10eb33198,scene-0061,39586f9d59004284a7114a68825e8eec,356d81f38dd9473ba590f39e266f54e5,../data/sets/nuscenes\samples/RADAR_FRONT/n015...,../data/sets/nuscenes\samples/CAM_FRONT/n015-2...,409.850669,1176.970211,0


In [13]:
final_df = final_df.merge(
    sample_pcd_image_ref_df[['sample_token', 'next_token', 'radar_pcd_file', 
                             'image_file', 'camera_pose_x',	'camera_pose_y', 'camera_pose_z']],
    on='sample_token',
    how='left'
)
final_df.head(2)

,sample_token,x_global,y_global,z_global,radar_token,radar_x,radar_y,radar_z,radar_cam_x,radar_cam_y,...,rear_right_rpm,can_utime,vehicle_speed,yaw_rate,next_token,radar_pcd_file,image_file,camera_pose_x,camera_pose_y,camera_pose_z
0,ca9a282c9e77460f8360f564131a8af5,400.178422,1170.502939,0.495531,37091c75b9704e0daa829ba56dfa0906,10.0,-6.9,0,6.948967,0.96407,...,276.1791,1532402928127800,31.44,0.6,39586f9d59004284a7114a68825e8eec,../data/sets/nuscenes\samples/RADAR_FRONT/n015...,../data/sets/nuscenes\samples/CAM_FRONT/n015-2...,411.419986,1181.197176,0
1,ca9a282c9e77460f8360f564131a8af5,400.178422,1170.502939,0.495531,37091c75b9704e0daa829ba56dfa0906,10.0,-6.9,0,6.948967,0.96407,...,276.1791,1532402928127800,31.44,0.6,39586f9d59004284a7114a68825e8eec,../data/sets/nuscenes\samples/RADAR_FRONT/n015...,../data/sets/nuscenes\samples/CAM_FRONT/n015-2...,411.419986,1181.197176,0


In [14]:
final_df.shape

(3860, 64)

In [15]:
final_df.to_excel('Filtered_PCD_Annotations.xlsx', index=False)

In [16]:
final_df.columns.to_list()

['sample_token',
 'x_global',
 'y_global',
 'z_global',
 'radar_token',
 'radar_x',
 'radar_y',
 'radar_z',
 'radar_cam_x',
 'radar_cam_y',
 'radar_cam_z',
 'dyn_prop',
 'cluster_id',
 'rcs',
 'vx',
 'vy',
 'vx_comp',
 'vy_comp',
 'is_quality_valid',
 'ambig_state',
 'x_rms',
 'y_rms',
 'invalid_state',
 'pdh0',
 'vx_rms',
 'vy_rms',
 'image_coord_x',
 'image_coord_y',
 'depth',
 'depth_patch',
 'scene_name',
 'scene_token',
 'timestamp',
 'annotation_token',
 'category',
 'instance_token',
 'center_x',
 'center_y',
 'center_z',
 'length',
 'width',
 'height',
 'rotation_w',
 'rotation_x',
 'rotation_y',
 'rotation_z',
 'yaw',
 'num_lidar_pts',
 'num_radar_pts',
 'distance',
 'can_timestamp',
 'brake',
 'brake_switch',
 'rear_left_rpm',
 'rear_right_rpm',
 'can_utime',
 'vehicle_speed',
 'yaw_rate',
 'next_token',
 'radar_pcd_file',
 'image_file',
 'camera_pose_x',
 'camera_pose_y',
 'camera_pose_z']

In [17]:
# Create a renamed copy of the original DataFrame to act as "B"
df_a = final_df.copy()
df_b = final_df.copy()

df_a = df_a.add_prefix("a_")  # So B.sample_token becomes next_sample_token
df_b = df_b.add_prefix("b_")  # So B.sample_token becomes next_sample_token

# Perform the join: A.next_sample_token == B.sample_token AND A.instance_token == B.instance_token
df_ab = pd.merge(
    df_a,         # A
    df_b,         # B
    left_on=["a_next_token", "a_instance_token"],
    right_on=["b_sample_token", "b_instance_token"],
    how="inner"
)


In [18]:
df_ab.shape

(5450, 128)

In [19]:
df_ab = df_ab.drop_duplicates()
df_ab.shape

(5450, 128)

In [20]:
df_ab.to_excel('Filtered_PCD_Annotations_AB.xlsx', index=False)